<h4> File responsible for collect overview collection status

In [1]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os 
import re


In [2]:
load_dotenv()
slug_collection = 'boredapeyachtclub'
api_key = os.getenv('api_key')

url_stats = f'https://api.opensea.io/api/v2/collections/{slug_collection}/stats'
url_nft_overview = f"https://api.opensea.io/api/v2/collections/{slug_collection}"
headers = {
    "accept" : "application/json",
    "x-api-key": api_key
}

<h5> Request stats

In [3]:
try:
    response_stats = requests.get(url_stats, headers=headers).json()
    #response.raise_for_status()
    print(response_stats)
except requests.exceptions.RequestException as e:
    print(f"Erro na requisição: {e}")
    

{'total': {'volume': 1597939.8636721722, 'sales': 55073, 'average_price': 29.014941326460736, 'num_owners': 5376, 'market_cap': 133248.89262871238, 'floor_price': 13.1, 'floor_price_symbol': 'ETH'}, 'intervals': [{'interval': 'one_day', 'volume': 104.94799, 'volume_diff': -63.0, 'volume_change': -0.8837066320552495, 'sales': 8, 'sales_diff': -0.8873239436619719, 'average_price': 13.11849875}, {'interval': 'seven_day', 'volume': 2665.7775858500027, 'volume_diff': 118.0, 'volume_change': 1.3469728034118018, 'sales': 200, 'sales_diff': 0.0, 'average_price': 13.328887929250014}, {'interval': 'thirty_day', 'volume': 6098.88812555, 'volume_diff': 195.0, 'volume_change': 0.48316111557592806, 'sales': 454, 'sales_diff': 0.0, 'average_price': 13.433674285352422}]}


<h5> Request nft collection overview

In [4]:
try:
    response_overview = requests.get(url_nft_overview, headers=headers).json()
    #response.raise_for_status()
    print(response_overview)
except requests.exceptions.RequestException as e:
    print(f"Erro na requisição: {e}")
    

{'collection': 'boredapeyachtclub', 'name': 'Bored Ape Yacht Club', 'description': 'The Bored Ape Yacht Club is a collection of 10,000 unique Bored Ape NFTs— unique digital collectibles living on the Ethereum blockchain. Your Bored Ape doubles as your Yacht Club membership card, and grants access to members-only benefits, the first of which is access to THE BATHROOM, a collaborative graffiti board. Future areas and perks can be unlocked by the community through roadmap activation. Visit www.BoredApeYachtClub.com for more details.', 'image_url': 'https://i.seadn.io/gae/Ju9CkWtV-1Okvf45wo8UctR-M9He2PjILP0oOvxE89AyiPPGtrR3gysu1Zgy0hjd2xKIgjJJtWIc0ybj4Vd7wv8t3pxDGHoJBzDB?w=500&auto=format', 'banner_image_url': 'https://i.seadn.io/gae/i5dYZRkVCUK97bfprQ3WXyrT9BnLSZtVKGJlKQ919uaUB0sxbngVCioaiyu9r6snqfi2aaTyIvv6DHm4m2R3y7hMajbsv14pSZK8mhs?w=500&auto=format', 'owner': '0xa858ddc0445d8131dac4d1de01f834ffcba52ef1', 'safelist_status': 'verified', 'category': 'pfps', 'is_disabled': False, 'is_nsfw

<p> Creating dataframe with collection stats

In [5]:
df_total_status = pd.json_normalize(response_stats['total'])
#$2,637.70 USD (ETH price 2025-02-09)
eth_price = 2637.7
df_total_status['volume - USD'] = (df_total_status['volume'] * eth_price).round(2)
df_total_status ['floor_price - USD'] = (df_total_status['floor_price'] * eth_price).round(2)
df_total_status['collection'] = slug_collection
display(df_total_status)

,volume,sales,average_price,num_owners,market_cap,floor_price,floor_price_symbol,volume - USD,floor_price - USD,collection
0,1.597940e+06,55073,29.014941,5376,133248.892629,13.1,ETH,4.214886e+09,34553.87,boredapeyachtclub


<p> Creating dataframe with collection overview

In [6]:
df_overview = pd.json_normalize(response_overview)
display(df_overview)

,collection,name,description,image_url,banner_image_url,owner,safelist_status,category,is_disabled,is_nsfw,...,editors,fees,payment_tokens,total_supply,created_date,rarity.strategy_id,rarity.strategy_version,rarity.calculated_at,rarity.max_rank,rarity.tokens_scored
0,boredapeyachtclub,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,https://i.seadn.io/gae/Ju9CkWtV-1Okvf45wo8UctR...,https://i.seadn.io/gae/i5dYZRkVCUK97bfprQ3WXyr...,0xa858ddc0445d8131dac4d1de01f834ffcba52ef1,verified,pfps,False,False,...,"[0xa858ddc0445d8131dac4d1de01f834ffcba52ef1, 0...","[{'fee': 0.5, 'recipient': '0x0000a26b00c1f0df...","[{'symbol': 'ETH', 'address': '0x0000000000000...",9997,2021-04-22,openrarity,0.4.3b0,2022-09-21T12:18:03.321743,10000,10000


In [7]:
df_overview_status = df_total_status.merge(df_overview, on="collection",how="left")


In [8]:
def sanitize_filename(name):
    return re.sub(r'[\\/:*?"<>|]', '', name)

safe_slug = sanitize_filename(slug_collection)

datasource_dir = "datasource"

if not os.path.exists(datasource_dir):
    os.makedirs(datasource_dir)

dataframe_json = df_overview_status.to_json(safe_slug +"_overview_status.json")